# Trading Analysis based on RSI and MACD

Trading based on RSI (Relative Strength Index) and MACD (Moving Average Convergence Divergene) are among the most popular techniques. 

MACD is a trend-following indicator that generates buy and sell signal based on movement of two moving average. The signal line is calculated by 7-day MA. The MACD line is a 25-day MA When the signal line cross above the MACD line, it generates a buy signal. When signal cross below the MACD line, it generates a sell signal. However, according to "Technical Analysis of Financial Markets" by John J.Murphy, this techniques does not work well in a trendless market. That's when other oscillators like RSI comes into play.

Trader uses RSI to identify when a commodity/asset is oversold or overbought territory. RSI is calculated based on the average gain and average loss in a period of time. In this analysis, we'll analyse to see if RSI or MACD is a better technique (used through multiple randomized period of time) when it comes to trading cryptocurrency (we use Ethereum in this case)

In [1]:
#pip install python-binance

In [2]:
#Import library

import pandas as pd
import numpy as np
import datetime
import plotly.graph_objects as go
from binance.client import Client

#### 1.Clean Data

In [3]:
# API = "Insert your API here"
API_key = "Insert your API here"
API_secret = "Insert your API here"
CurrencyPair = "ETHUSDT"
StartTime = "1 Sep, 2018"
EndTime = "13 Sep, 2019"

#Specify quantity order in each trade and initial capital
quantity_order = 100
initial_capital = 100000 

#Clean data function

def clean_data(API_key, API_secret, CurrencyPair, StartTime, EndTime):
    client = Client(API_key, API_secret)
    data = client.get_historical_klines(CurrencyPair, Client.KLINE_INTERVAL_1DAY, StartTime, EndTime)
    df = pd.DataFrame(data, columns =["Open time", "Open", "High", "Low", "Close", "Volume", "Close time",
                                  "Quote asset volume", "Number of trades", "Taker buy base asset volume", 
                                  "Taker buy quote asset volume", "Ignore"])
    df = df.astype({'Open': 'float', 'High': 'float', 'Low': 'float', 'Close': 'float', 'Volume': 'float'})
    
    for col in ['Open time', 'Close time']:
        df[col] = df[col].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.0))
        
    df.drop(columns=['Ignore'],inplace=True)
    return df


#### 2.Trade RSI

In [4]:
def trade_RSI(API_key, API_secret, CurrencyPair, StartTime, EndTime, initial_capital, quantity_order = 100):
    df =  clean_data(API_key, API_secret, CurrencyPair, StartTime, EndTime)
    
    df['Net diff'] = df['Close'].diff()
    df['Net Gain'] = df['Net diff'].apply(lambda x: x if x > 0 else 0 )
    df['Net Loss'] =  df['Net diff'].apply(lambda x: -x if x < 0 else 0 )
    df['Average Gain'] = df['Net Gain'].rolling(14).mean()
    df['Average Loss'] = df['Net Loss'].rolling(14).mean()
    df['Relative Strength'] = df['Average Gain']/df['Average Loss']
    df['RSI'] = 100-(100/(1+df['Relative Strength']))
    
    df.drop(columns=['Net diff','Net Gain', 'Net Loss', 
                 'Average Gain', 'Average Loss', 'Relative Strength'],inplace=True)
    
    
    trade = df[['Open time', 'Open', 'Close', 'High', 'Low', 'RSI']].copy()
    trade = trade.dropna()
    trade['Quantity_order'] = 0
    trade['Holding_quantity'] = 0
    
    capital = initial_capital
    holding = 0
    
    for i in trade.index:
        if ((trade.at[i, 'RSI'] < 30)  and (capital >= trade.at[i,'Close']*quantity_order)):
            trade.at[i, 'Quantity_order'] = quantity_order
            trade.at[i,'Buy/Sell'] = "Buy"
            holding += quantity_order
            trade.at[i,'Holding_quantity'] = holding
            capital -= trade.at[i,'Close']*trade.at[i,'Quantity_order']
        elif trade.at[i, 'RSI'] > 70:
            if abs(holding) >= 0:
                trade.at[i,'Buy/Sell'] = "Sell"
                trade.at[i, 'Quantity_order'] = -holding
                capital -= trade.at[i,'Close']*trade.at[i,'Quantity_order']
                trade.at[i,'Holding_quantity'] = 0
                holding = 0
            

    trade['Order Value'] = np.where(trade['Quantity_order'] !=0, trade['Close']*trade['Quantity_order'] ,0)
    trade['Balance'] = initial_capital - trade['Order Value'].cumsum()
    trade = trade[trade['Quantity_order'] != 0]

    #Calculate performance based on last sell
    
    i = len(trade)-1
    if (i >= 2):
        while (trade.iloc[i]['Holding_quantity'] != 0):
            trade = trade.drop(trade.index[i])
            i -= 1
        
    return (-trade['Order Value'].sum(), -trade['Order Value'].sum()/initial_capital, trade)
    

In [5]:
Profit, ROI, trade = trade_RSI(API_key, 
          API_secret, 
          "ETHUSDT", 
          "1 Jan, 2018",
          "1 Jan, 2022",
          100000
         )

trade

,Open time,Open,Close,High,Low,RSI,Quantity_order,Holding_quantity,Buy/Sell,Order Value,Balance
41,2018-02-11 11:00:00,851.15,808.99,857.00,768.28,28.447429,100,100,Buy,80899.0,19101.0
49,2018-02-19 11:00:00,911.00,938.11,956.22,906.04,70.188445,-100,0,Sell,-93811.0,112912.0
67,2018-03-09 11:00:00,695.48,723.71,727.92,640.00,29.522529,100,100,Buy,72371.0,40541.0
87,2018-03-29 11:00:00,446.07,384.00,449.43,380.00,10.197217,100,200,Buy,38400.0,2141.0
101,2018-04-12 10:00:00,429.90,495.67,499.00,412.88,73.012406,-200,0,Sell,-99134.0,101275.0
138,2018-05-19 10:00:00,693.26,697.00,717.80,682.31,29.151065,100,100,Buy,69700.0,31575.0
224,2018-08-13 10:00:00,317.96,283.81,321.99,280.68,8.032324,100,200,Buy,28381.0,3194.0
353,2018-12-20 11:00:00,98.73,114.05,116.00,97.88,71.346546,-200,0,Sell,-22810.0,26004.0
382,2019-01-18 11:00:00,121.56,119.17,121.98,117.66,26.002341,100,100,Buy,11917.0,14087.0
383,2019-01-19 11:00:00,119.16,122.60,125.29,119.14,28.684248,100,200,Buy,12260.0,1827.0


#### 3.Trade MACD

In [6]:
def trade_MACD(API_key, API_secret, CurrencyPair, StartTime, EndTime, initial_capital, signal = 7, MACD = 25, quantity_order = 100):
    df =  clean_data(API_key, API_secret, CurrencyPair, StartTime, EndTime)
    
    
    df['Signal Line'] = df['Close'].rolling(window=signal).mean()
    df['MACD'] = df['Close'].rolling(window=MACD).mean()
    df['Signal'] = np.where(df['Signal Line'] < df['MACD'], 1, 0)
    df['Position'] = df['Signal'].diff()
    
    df[df['Position'] !=0]
    
    trade = df[['Open time', 'Open', 'Close', 'High', 'Low', 'Signal Line', 'MACD', 'Signal', 'Position']].copy()
    trade['Order Value'] = 0
    trade['Buy/Sell'] =""
    trade['Quantity_order'] = 0
    trade['Holding_quantity'] = 0
    trade = trade.dropna()
    
    capital = initial_capital
    holding = 0
    for i in trade.index:
        if ((trade.at[i, 'Position'] == -1)  and (capital >= trade.at[i,'Close']*quantity_order)):
            trade.at[i, 'Quantity_order'] = quantity_order
            trade.at[i,'Buy/Sell'] = "Buy"
            holding += quantity_order
            trade.at[i,'Holding_quantity'] = holding
            capital -= trade.at[i,'Close']*trade.at[i,'Quantity_order']
        elif trade.at[i, 'Position'] == 1:
            if abs(holding) >= 0:
                trade.at[i,'Buy/Sell'] = "Sell"
                trade.at[i, 'Quantity_order'] = -holding
                capital -= trade.at[i,'Close']*trade.at[i,'Quantity_order']
                trade.at[i,'Holding_quantity'] = 0
                holding = 0
                
    trade.drop(columns=['Signal','Position'],inplace=True)
    trade['Order Value'] = np.where(trade['Quantity_order'] !=0, trade['Close']*trade['Quantity_order'] ,0)
    trade['Balance'] = initial_capital - trade['Order Value'].cumsum()
    trade = trade[trade['Quantity_order'] != 0]

    #Calculate performance based on last sell
    
    i = len(trade)-1
    
    if (i >= 2):
        while (trade.iloc[i]['Holding_quantity'] != 0):
            trade = trade.drop(trade.index[i])
            i -= 1
        
    return(-trade['Order Value'].sum(), -trade['Order Value'].sum()/initial_capital, trade)

In [7]:
Profit, ROI, trade = trade_MACD(API_key, 
          API_), 
          "ETHUSDT", 
          "01 Sep, 2018",
          "13 Sep, 2019",
          100000
         )
print(Profit)
print(ROI)
trade

8019.999999999996
0.08019999999999997


,Open time,Open,Close,High,Low,Signal Line,MACD,Order Value,Buy/Sell,Quantity_order,Holding_quantity,Balance
66,2018-11-06 11:00:00,210.64,220.74,221.25,209.11,206.408571,205.7012,22074.0,Buy,100,100,77926.0
75,2018-11-15 11:00:00,187.43,184.35,190.00,170.99,204.438571,205.5272,-18435.0,Sell,-100,0,96361.0
112,2018-12-22 11:00:00,107.67,115.20,115.50,105.38,102.108571,100.5656,11520.0,Buy,100,100,84841.0
134,2019-01-13 11:00:00,123.35,114.16,124.25,111.87,133.164286,133.6152,-11416.0,Sell,-100,0,96257.0
163,2019-02-11 11:00:00,125.12,120.89,125.28,119.24,114.385714,113.0232,12089.0,Buy,100,100,84168.0
185,2019-03-05 11:00:00,125.68,136.72,137.84,125.19,133.030000,133.3516,-13672.0,Sell,-100,0,97840.0
199,2019-03-19 11:00:00,137.62,138.55,139.38,136.97,136.591429,135.9176,13855.0,Buy,100,100,83985.0
237,2019-04-26 10:00:00,154.70,157.95,162.56,151.40,165.514286,166.7468,-15795.0,Sell,-100,0,99780.0
247,2019-05-06 10:00:00,163.05,173.00,178.05,157.55,164.520000,164.5184,17300.0,Buy,100,100,82480.0
279,2019-06-07 10:00:00,248.77,248.77,253.28,243.46,252.524286,252.8888,-24877.0,Sell,-100,0,107357.0


In [8]:
# Default basedate is BTC's first launch. 
# This function randomize trading period through a set of time

import random

def randomize__dates(baseDate = 3, baseMonth = 1, baseYear = 2009, period = 365):
    lower_constraint = datetime.datetime(baseYear, baseMonth, baseDate)
    upper_constraint = datetime.datetime.today() - datetime.timedelta(days = period)
    
    delta = upper_constraint - lower_constraint
    int_delta = delta.days
    int_delta = random.randrange(int_delta)
    
    start_date = lower_constraint + datetime.timedelta(days = int_delta)
    end_date = start_date + datetime.timedelta(days = period)
    
    start = start_date.strftime("%d %b, %Y") 
    end = end_date.strftime("%d %b, %Y") 
 
    return  str(start), str(end)

### Testing through different time frame

In [9]:
# API = "Insert your API here"
API_key = "Insert your API here"
API_secret = "Insert your API here"
CurrencyPair = "ETHUSDT"

#Specify quantity order in each trade and initial capital
quantity_order = 100
initial_capital = 100000 

#### Testing RSI on randomized one-year periods

In [10]:

#Setting variable for testing
number_period = 10

sum_profit = 0
total = 0

for i in range(0,number_period):
    start, end = randomize__dates()
    profit, ROI, trade = trade_RSI(API_key, 
                                    API_secret, CurrencyPair, 
                                    start,end,initial_capital) 
    sum_profit += profit
    total += ROI
    
average_profit = sum_profit/number_period
average_ROI = total/number_period

In [11]:
print("Average Profit: " + str(average_profit))
print("Average ROI: "+ "{0:.0%}".format(average_ROI))

Average Profit: -15216.5
Average ROI: -15%


#### Testing MACD on randomized one-year periods

In [12]:
sum_profit = 0
total = 0

for i in range(0,number_period):
    start, end = randomize__dates()
    profit, ROI, trade = trade_MACD(API_key, 
                                    API_secret, CurrencyPair, 
                                    start,end,initial_capital) 
    sum_profit += profit
    total += ROI
    
average_profit = sum_profit/number_period
average_ROI = total/number_period
    

In [13]:
print("Average Profit: " + str(average_profit))
print("Average ROI: "+ "{0:.0%}".format(average_ROI))

Average Profit: 11916.5
Average ROI: 12%


### 3. Conclusion

Overall, after testing, MACD appears to be a better indicator than RSI when it comes to trading cyptocurrency. This might  due to the fact that cryptocurrency is speculative assets with high volatility, as a result, there're lots of impulsive price action instead of consolidation (trend-less) periods. 